In [128]:
import pandas as pd
import requests
import http.client
from dotenv import load_dotenv
import os
import json
import warnings
warnings.simplefilter(action='ignore')

load_dotenv()

TOKEN = os.getenv('TOKEN')
SECRET_KEY = os.getenv('key')
id_loja = os.getenv('IDLOJA')

In [126]:
##Reader Excel Files
base_precos = pd.read_csv(r"C:\api_vtex\preco_produtos.csv",sep=";")
tabela_1 = base_precos[base_precos['Trade Policy']==1]
tabela_1['Price'] = tabela_1['Price'].astype(float)

In [120]:
import http.client

conn = http.client.HTTPSConnection("api.vtex.com")

In [ ]:
#Atualiza Preços

def update_lists_prices_products(*args, **kwargs):
    '''
    
    payload = "{\"markup\":0,\"listPrice\":0,\"basePrice\":0,\"costPrice\":0,\"fixedPrices\":[{\"tradePolicyId\":\"1\",\"value\":50.5,\"listPrice\":50.5,\"minQuantity\":2,\"dateRange\":{\"from\":\"2021-12-30T22:00:00-03:00\",\"to\":\"2021-12-30T22:00:00-03:00\"}}]}"

    headers = {
        'Accept': "application/json",
        'Content-Type': "application/json",
        'X-VTEX-API-AppKey': f"{SECRET_KEY}",
        'X-VTEX-API-AppToken': f"{TOKEN}"
        }


    conn.request("put", f"/{id_loja}/pricing/prices/{itemId}", payload, headers)

    res = conn.getresponse()
    data = res.read()

    print(data.decode("utf-8"))
'''


update_lists_prices_products()


In [125]:

#Retorna tabela preços
def listing_table_prices(idloja):

    headers = {
    'Accept': "application/json",
    'Content-Type': "application/json",
    'X-VTEX-API-AppKey': f"{SECRET_KEY}",
    'X-VTEX-API-AppToken': f"{TOKEN}"
    }

    conn.request("get", f"/{idloja}/pricing/pipeline/catalog/1", headers=headers)

    res = conn.getresponse()
    data = res.read()
    jsons = json.loads(data.decode("utf-8"))
    try:
        dict_tables = {
            "tableinfos":jsons['rules'],
            "politicacomercial":jsons['tradePolicyId']}
       
        yield dict_tables
    except Exception as e:
        print(e)
    

dicts = listing_table_prices()
print(next(dicts))


{'tableinfos': [{'id': 0, 'context': {'categories': {}, 'brands': {}, 'stockStatuses': None, 'internalCategories': None, 'markupRange': None, 'dateRange': None}, 'percentualModifier': 10}], 'politicacomercial': '1'}


In [89]:
#Retorna Lista de produtos e preçs
def get_products_prices(produtoid,id_loja,idtabela=1):
    headers = {
        'Accept': "application/json",
        'Content-Type': "application/json",
        'X-VTEX-API-AppKey': f"{SECRET_KEY}",
        'X-VTEX-API-AppToken': f"{TOKEN}"
        }

    conn.request("get", f"/{id_loja}/pricing/prices/{produtoid}/fixed/{idtabela}", headers=headers)

    res = conn.getresponse()
    data = res.read()
    jsons = json.loads(data.decode("utf-8"))
    
    for js in jsons:
        product_dict = {}
        try:
            product_dict['PoliticaPreco'] = js.get('tradePolicyId',None)
            product_dict['Preco'] = float(js.get('value',None))
            
            print(product_dict)
        except Exception as e:
            print(e)
        
    
    

In [ ]:
product_id = tabela_1[['Trade Policy','SKU ID']].to_dict('records')
for product in product_id:
    get_products_prices(product['SKU ID'],product['Trade Policy'])
   

In [ ]:
#Lista inventario por produto
def listing_inventory_itens(sku,id_loja):
        headers = {
                'Accept': "application/json",
                'Content-Type': "application/json",
                'X-VTEX-API-AppKey': f"{SECRET_KEY}",
                'X-VTEX-API-AppToken': f"{TOKEN}"
                }

        conn.request("get", f"/{id_loja}/admin/inventory.com.br/api/logistics/pvt/inventory/items/{sku}/docks/", headers=headers)

        res = conn.getresponse()
        data = res.read()

        print(data.decode("utf-8"))
